<a href="https://colab.research.google.com/github/OdysseusPolymetis/digital_classics_course/blob/main/6_geolocalization_lat_gk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Géolocalisation avec Pleiades**

Pour ça nous allons avoir besoin essentiellement de deux moteurs : un moteur de lemmatisation (stanza), et un moteur de détection d'entités nommées. Suite à cela, nous allons utiliser les données de Pleiades pour géolocaliser les entités trouvées.

In [27]:
!pip install flair
!pip install stanza
!pip install transformers

Ici on importe les bibliothèques dont on aura besoin, et comme pour tout modèle de type transformer, il vaut mieux avoir une GPU. Et nous lui disons de mettre le modèle sur la GPU.
<br>Selon que vous aurez choisi un texte grec ou latin, vous aurez le modèle adéquat. Le modèle pour le latin sera sûrement moins bon que le modèle pour le grec. Pour le grec, on utilisera stanza, pour le latin on utilisera spacy (parce que c'est lui qui a le meilleur moteur de NER pour le latin).

C'est dans la cellule qui suit que vous devez préciser si vous faites du latin ou du grec, et donc `"la"` ou `"grc"`.

In [31]:
LANG = "la"

In [ ]:
from flair.data import Sentence
from flair.models import SequenceTagger
import torch
import flair

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Device utilisé :", DEVICE, "| Langue :", LANG)

flair.device = torch.device(DEVICE)
tagger = SequenceTagger.load("UGARIT/flair_grc_bert_ner")
tagger.to(DEVICE)

In [28]:
import unicodedata
from collections import defaultdict

import torch
import pandas as pd
import requests
import folium
from tqdm import tqdm

import stanza
import flair
from flair.data import Sentence
from flair.models import SequenceTagger

from transformers import pipeline

Ici nous allons chercher les données de Pleiades, et nous les dézippons.

In [ ]:
!wget https://atlantides.org/downloads/pleiades/gis/pleiades_gis_data.zip

In [ ]:
!unzip pleiades_gis_data.zip

Ici vous allez pouvoir mettre votre texte. Vous pouvez mettre du latin ou du grec.

In [39]:
from google.colab import files
uploaded = files.upload()

Saving lat0690-003.txt to lat0690-003.txt


Ici il va vous falloir mettre les fichiers qui vous intéressent. Normalement vous n'avez à changer que la première ligne, celle du nom de votre txt.

In [40]:
TEXT_PATH = "/content/lat0690-003.txt"
NAMES_CSV = "/content/data/gis/names.csv"
OUTPUT_MAP = "map_ancient_places.html"

Cette fonction vous permet de:


*   supprimer les diacritiques (accents)
*   mettre en minuscules



In [41]:
def normalize_name(name: str) -> str:

    if not isinstance(name, str):
        return ""
    return "".join(
        c
        for c in unicodedata.normalize("NFD", name)
        if unicodedata.category(c) != "Mn"
    ).lower()

Là c'est une fonction de translittération très simplifiée du grec vers l'alphabet latin, si jamais les données Pleiades ne contiennent pas le grec nativement.

In [42]:
def transliterate_greek_to_latin(s: str) -> str:

    s = normalize_name(s)
    table = {
        'α': 'a',  'β': 'b',  'γ': 'g',   'δ': 'd',
        'ε': 'e',  'ζ': 'z',  'η': 'e',   'θ': 'th',
        'ι': 'i',  'κ': 'k',  'λ': 'l',   'μ': 'm',
        'ν': 'n',  'ξ': 'x',  'ο': 'o',   'π': 'p',
        'ρ': 'r',  'σ': 's',  'ς': 's',   'τ': 't',
        'υ': 'u',  'φ': 'ph', 'χ': 'ch',  'ψ': 'ps',
        'ω': 'o',
    }
    return "".join(table.get(ch, ch) for ch in s)

Cette partie-là vous donne un aperçu de votre texte en cours.

In [43]:
with open(TEXT_PATH, encoding="utf-8") as f:
    text = f.read()

print("Longueur du texte :", len(text), "caractères")
print("Aperçu :")
print(text[:1000])

Longueur du texte : 443826 caractères
Aperçu :
       {AENEIS}         {LIBER I} Arma uirumque cano, Troiae qui primus ab oris Italiam fato profugus Lauiniaque uenit litora, multum ille et terris iactatus et alto ui superum, saeuae memorem Iunonis ob iram, multa quoque et bello passus, dum conderet urbem inferretque deos Latio; genus unde Latinum Albanique patres atque altae moenia Romae. Musa, mihi causas memora, quo numine laeso quidue dolens regina deum tot uoluere casus insignem pietate uirum, tot adire labores impulerit. tantaene animis caelestibus irae?     Vrbs antiqua fuit (Tyrii tenuere coloni) Karthago, Italiam contra Tiberinaque longe ostia, diues opum studiisque asperrima belli, quam Iuno fertur terris magis omnibus unam posthabita coluisse Samo. hic illius arma, hic currus fuit; hoc regnum dea gentibus esse, si qua fata sinant, iam tum tenditque fouetque. progeniem sed enim Troiano a sanguine duci audierat Tyrias olim quae uerteret arces; hinc populum late regem belloque s

La cellule suivante charge le modèle stanza adéquat en fonction de la langue que vous avez demandée au départ.

In [ ]:
stanza.download(LANG)
nlp = stanza.Pipeline(
    lang=LANG,
    processors="tokenize,lemma",
    use_gpu=(DEVICE == "cuda"),
    device=DEVICE
)

Ici on va utiliser deux types de moteurs NER, en fonction du grec ou du latin.

In [ ]:
if LANG == "grc":

    flair.device = torch.device(DEVICE)
    FLAIR_MODEL = "UGARIT/flair_grc_bert_ner"
    tagger_grc = SequenceTagger.load(FLAIR_MODEL)
    tagger_grc.to(DEVICE)
    print("Modèle Flair grec chargé :", FLAIR_MODEL)
    ner_pipe_la = None

elif LANG == "la":

    device_id = 0 if DEVICE == "cuda" else -1
    ner_pipe_la = pipeline(
        "token-classification",
        model="magistermilitum/roberta-multilingual-medieval-ner",
        device=device_id,
    )
    print("Pipeline NER latin (Transformers) initialisé.")
    tagger_grc = None

Cette fonction fusionne les sous-mots renvoyés par roberta-multilingual-medieval-ner en entités complètes, en utilisant les offsets de caractères (logique proche de la TextProcessor du model card).

    raw_ents : liste de dicts du pipeline HF
               (avec 'entity' ou 'entity_group' ou 'label', 'word', 'start', 'end')
    sent_text : texte brut de la phrase

Retourne une liste de tuples : (surface, base_label, start, end) où base_label est par ex. 'LOC' ou 'PERS', et surface est le mot complet extrait du texte (par ex. 'Troiae', 'Lavinia', etc.).

In [106]:
def merge_latin_entities(raw_ents, sent_text: str):

    if not raw_ents:
        return []

    sample = raw_ents[0]
    if "entity" in sample:
        label_key = "entity"
    elif "entity_group" in sample:
        label_key = "entity_group"
    else:
        label_key = "label"

    merged_spans = []
    current = None

    for ent in raw_ents:
        full_label = ent[label_key]
        base_label = full_label.split("-")[-1]
        start, end = ent["start"], ent["end"]

        if current is None:
            current = {"label": base_label, "start": start, "end": end}
        else:
            if base_label == current["label"] and start == current["end"]:
                current["end"] = end
            else:
                merged_spans.append(current)
                current = {"label": base_label, "start": start, "end": end}

    if current is not None:
        merged_spans.append(current)

    merged_entities = []
    n = len(sent_text)

    for span in merged_spans:
        start = span["start"]
        end = span["end"]
        label = span["label"]

        while start > 0 and sent_text[start - 1].isalpha():
            start -= 1

        while end < n and sent_text[end].isalpha():
            end += 1

        surface = sent_text[start:end].strip()
        if surface:
            merged_entities.append((surface, label, start, end))


    return merged_entities

Ici on va lancer l'analyse globale du texte.

In [44]:
doc = nlp(text)
print("Nombre de phrases détectées par Stanza :", len(doc.sentences))

Nombre de phrases détectées par Stanza : 4494


Essaie de retrouver un lemme Stanza pour une entité NER.

*   gère les entités sur un seul mot (Troiae)
*   gère les entités multi-mots (ex. 'oris Italiam', 'deos Latio') en testant chaque morceau, en commençant par le dernier (souvent le vrai lieu).

In [101]:
def find_lemma_by_text(entity_text: str, stanza_sentence) -> str | None:

    if not entity_text:
        return None

    parts = entity_text.split()
    candidate_texts = []

    if len(parts) == 1:
        candidate_texts = [entity_text]
    else:
        candidate_texts = list(reversed(parts))

    for cand in candidate_texts:
        ent_norm = normalize_name(cand)
        if not ent_norm:
            continue

        # 1) égalité stricte
        for word in stanza_sentence.words:
            tok_norm = normalize_name(word.text)
            if tok_norm == ent_norm:
                return word.lemma.lower()

        # 2) entité incluse dans le token
        for word in stanza_sentence.words:
            tok_norm = normalize_name(word.text)
            if ent_norm in tok_norm:
                return word.lemma.lower()

        # 3) token inclus dans l'entité
        for word in stanza_sentence.words:
            tok_norm = normalize_name(word.text)
            if tok_norm and tok_norm in ent_norm:
                return word.lemma.lower()

    return None

Ici on:
Construit un dictionnaire :
        { lemme_lieu (LOC) -> set({ lemme_personne1, lemme_personne2, ... }) }

    - doc  : document Stanza (grec ou latin)
    - lang : "grc" (grec) ou "la" (latin)
    - window : (pas utilisé ici, on associe par phrase)
    - batch_size : taille des batchs pour Flair (grec)

    Dépend de variables globales :
    - tagger_grc   (Flair NER grec)         si lang == "grc"
    - ner_pipe_la  (pipeline HF latin NER)  si lang == "la"
    - find_lemma_by_text (pour retrouver le lemme Stanza)
    - merge_latin_entities (pour fusionner les spans du modèle latin)

In [104]:
def build_loc_to_pers(doc, lang: str, window: int = 100, batch_size: int = 32):

    loc_to_pers = defaultdict(set)

    if lang == "grc":
        if tagger_grc is None:
            raise RuntimeError("tagger_grc (Flair grec) n'est pas initialisé.")

        flair_sentences = [Sentence(s.text) for s in doc.sentences]
        n = len(flair_sentences)
        print("Grec : nombre de phrases pour Flair :", n)

        for i in tqdm(range(0, n, batch_size), desc="NER grec (Flair)"):
            batch_flair = flair_sentences[i:i + batch_size]
            batch_stanza = doc.sentences[i:i + batch_size]

            tagger_grc.predict(batch_flair, mini_batch_size=batch_size)

            for flair_sentence, stanza_sentence in zip(batch_flair, batch_stanza):
                loc_texts = []
                per_texts = []

                for ent in flair_sentence.get_spans("ner"):
                    if ent.tag == "LOC":
                        loc_texts.append(ent.text)
                    elif ent.tag == "PER":
                        per_texts.append(ent.text)

                for loc_text in loc_texts:
                    loc_lemma = find_lemma_by_text(loc_text, stanza_sentence)
                    if not loc_lemma:
                        continue
                    for per_text in per_texts:
                        per_lemma = find_lemma_by_text(per_text, stanza_sentence)
                        if per_lemma:
                            loc_to_pers[loc_lemma].add(per_lemma)

    elif lang == "la":
        if ner_pipe_la is None:
            raise RuntimeError("ner_pipe_la (pipeline NER latin) n'est pas initialisé.")

        print("Latin : NER phrase par phrase (Transformers + merge_latin_entities).")
        for stanza_sentence in tqdm(doc.sentences, desc="NER latin (Transformers)"):
            sent_text = stanza_sentence.text

            raw_ents = ner_pipe_la(sent_text)

            if not raw_ents:
                continue
            merged_ents = merge_latin_entities(raw_ents, sent_text)

            loc_texts = []
            per_texts = []

            for surface, base_label, start, end in merged_ents:
                if base_label == "LOC":
                    loc_texts.append(surface)
                elif base_label in {"PERS", "PER", "PERSON"}:
                    per_texts.append(surface)

            for loc_text in loc_texts:
                loc_lemma = find_lemma_by_text(loc_text, stanza_sentence)
                if not loc_lemma:
                    continue
                for per_text in per_texts:
                    per_lemma = find_lemma_by_text(per_text, stanza_sentence)
                    if per_lemma:
                        loc_to_pers[loc_lemma].add(per_lemma)

    else:
        raise ValueError(f"Langue non gérée : {lang} (attendu 'grc' ou 'la')")

    return loc_to_pers

In [ ]:
loc_to_pers = build_loc_to_pers(doc, LANG, window=100, batch_size=32)

In [ ]:
for loc, pers_set in list(loc_to_pers.items()):
    print(loc, ":", pers_set)

Ici on nettoie un peu le csv de Pleiades.

In [ ]:
names_df = pd.read_csv(NAMES_CSV)

names_df["attested_norm"] = names_df["attested_form"].apply(normalize_name)

for col in ["romanized_form_1", "romanized_form_2", "romanized_form_3"]:
    if col in names_df.columns:
        names_df[col + "_norm"] = names_df[col].fillna("").apply(normalize_name)

names_df.head()

Retourne un place_id Pleiades pour un lemme de lieu, selon la langue.

    - grec ("grc") : match direct sur les formes grecques, puis translittération grec -> latin
    - latin ("la") : on ignore un éventuel -us final et on cherche le "stem" comme sous-chaîne
                     dans attested_norm (d'abord en latin, puis toutes langues).

In [109]:
def find_place_id_for_loc(loc_lemma: str, names_df: pd.DataFrame, lang: str) -> int | None:

    lemma_norm = normalize_name(loc_lemma)


    if lang == "grc":
        greek_mask = names_df["language_tag"] == "grc"
        match_greek = names_df[greek_mask & (names_df["attested_norm"] == lemma_norm)]
        if not match_greek.empty:
            return match_greek.iloc[0]["place_id"]

        latin_guess = transliterate_greek_to_latin(loc_lemma)
        roman_cols = [c for c in names_df.columns
                      if c.endswith("_norm") and "romanized_form" in c]

        for col in roman_cols:
            match_rom = names_df[names_df[col] == latin_guess]
            if not match_rom.empty:
                return match_rom.iloc[0]["place_id"]

        match_attested = names_df[names_df["attested_norm"] == latin_guess]
        if not match_attested.empty:
            return match_attested.iloc[0]["place_id"]

        return None

    elif lang == "la":
        stem = lemma_norm
        if stem.endswith("us"):
            stem = stem[:-2]

        if len(stem) < 3:
            stem = lemma_norm

        latin_mask = names_df["language_tag"] == "la"
        latin_df = names_df[latin_mask]

        match_latin = latin_df[latin_df["attested_norm"].str.contains(stem, na=False)]
        if not match_latin.empty:
            return match_latin.iloc[0]["place_id"]

        match_any = names_df[names_df["attested_norm"].str.contains(stem, na=False)]
        if not match_any.empty:
            return match_any.iloc[0]["place_id"]

        return None

    else:
        return None

In [ ]:
for loc in list(loc_to_pers.keys()):
    pid = find_place_id_for_loc(loc, names_df, LANG)
    print(loc, "->", pid)

Ici on crée une carte Folium à partir de :
        loc_to_pers : { lemme_lieu -> { pers_1, pers_2, ... } }
        names_df    : données Pleiades (names.csv)
        lang        : "grc" ou "la"

    - Fond de carte antique (DARE / Imperium Romanum)
    - Fond moderne OSM en option
    - Marqueurs personnalisés avec popup (lieu + personnages)

In [111]:
def make_map_from_loc_to_pers(loc_to_pers, names_df: pd.DataFrame, lang: str) -> folium.Map:


    m = folium.Map(location=[37.9838, 23.7275], zoom_start=6, tiles=None)

    folium.TileLayer(
        tiles="https://dh.gu.se/tiles/imperium/{z}/{x}/{y}.png",
        attr="DARE / Johan Åhlfeldt, CC-BY",
        name="Fond antique (DARE)",
        overlay=False,
        control=True,
        max_zoom=11,
        min_zoom=4,
    ).add_to(m)

    folium.TileLayer(
        "OpenStreetMap",
        name="Fond moderne (OSM)",
        overlay=False,
        control=True,
    ).add_to(m)

    for loc_lemma, pers_set in loc_to_pers.items():
        pid = find_place_id_for_loc(loc_lemma, names_df, lang)
        if pid is None:
            continue

        url = f"http://pleiades.stoa.org/places/{pid}/json"
        try:
            pleiades_data = requests.get(url, timeout=10).json()
        except Exception as e:
            print(f"[WARN] Erreur pour place_id {pid} ({loc_lemma}) :", e)
            continue

        repr_point = pleiades_data.get("reprPoint")
        if not repr_point:
            continue

        lon, lat = repr_point

        popup_html = f"""
        <b>{loc_lemma}</b><br>
        <i>place_id</i> : {pid}<br>
        <b>Personnages :</b> {', '.join(sorted(pers_set))}
        """

        folium.Marker(
            location=[lat, lon],
            popup=popup_html,
            icon=folium.Icon(
                icon="university",
                prefix="fa",
                color="darkblue",
            ),
        ).add_to(m)

    folium.LayerControl().add_to(m)
    return m

On crée la carte

In [112]:
m = make_map_from_loc_to_pers(loc_to_pers, names_df, LANG)

[WARN] Erreur pour place_id 570248 (eurotus) : Response ended prematurely
[WARN] Erreur pour place_id 501434 (dardanus) : Response ended prematurely


On la sauvegarde

In [113]:
m.save(OUTPUT_MAP)

Puis on recrée un csv : on exporte dans un CSV les données utilisées pour la carte :
* lemme du lieu
* place_id Pleiades
* coordonnées (lat, lon)
* personnages associés (séparés par ';')
* langue (grc/la)

In [114]:
def export_map_data_to_csv(loc_to_pers, names_df: pd.DataFrame, lang: str, csv_path: str):

    rows = []

    for loc_lemma, pers_set in loc_to_pers.items():
        pid = find_place_id_for_loc(loc_lemma, names_df, lang)
        if pid is None:
            continue

        url = f"http://pleiades.stoa.org/places/{pid}/json"
        try:
            data = requests.get(url, timeout=10).json()
        except Exception as e:
            print(f"[WARN] Erreur pour place_id {pid} ({loc_lemma}) :", e)
            continue

        repr_point = data.get("reprPoint")
        if not repr_point:
            continue

        lon, lat = repr_point

        rows.append({
            "loc_lemma": loc_lemma,
            "place_id": pid,
            "lat": lat,
            "lon": lon,
            "persons": ";".join(sorted(pers_set)) if pers_set else "",
            "n_persons": len(pers_set),
            "lang": lang,
        })

    df = pd.DataFrame(rows)
    df.to_csv(csv_path, index=False, encoding="utf-8")
    print(f"✔ CSV exporté : {csv_path} ({len(df)} lignes)")
    return df

In [ ]:
csv_path = f"map_data_{LANG}.csv"
df_export = export_map_data_to_csv(loc_to_pers, names_df, LANG, csv_path)
df_export.head()

Et là on va pouvoir refaire une carte à partir du nouveau CSV !

In [ ]:
def make_map_from_csv(csv_path: str, output_html: str | None = None) -> folium.Map:

    df = pd.read_csv(csv_path)

    m = folium.Map(location=[37.9838, 23.7275], zoom_start=6, tiles=None)

    folium.TileLayer(
        tiles="https://dh.gu.se/tiles/imperium/{z}/{x}/{y}.png",
        attr="DARE / Johan Åhlfeldt, CC-BY",
        name="Fond antique (DARE)",
        overlay=False,
        control=True,
        max_zoom=11,
        min_zoom=4,
    ).add_to(m)

    folium.TileLayer(
        "OpenStreetMap",
        name="Fond moderne (OSM)",
        overlay=False,
        control=True,
    ).add_to(m)

    for _, row in df.iterrows():
        loc_lemma = row.get("loc_lemma", "")
        pid = row.get("place_id", "")
        lat = row.get("lat", None)
        lon = row.get("lon", None)
        persons_str = row.get("persons", "")

        if pd.isna(lat) or pd.isna(lon):
            continue

        persons_display = persons_str if isinstance(persons_str, str) and persons_str else "—"

        popup_html = f"""
        <b>{loc_lemma}</b><br>
        <i>place_id</i> : {pid}<br>
        <b>Personnages :</b> {persons_display}
        """

        folium.Marker(
            location=[lat, lon],
            popup=popup_html,
            icon=folium.Icon(
                icon="university",
                prefix="fa",
                color="darkblue",
            ),
        ).add_to(m)

    folium.LayerControl().add_to(m)

    if output_html is not None:
        m.save(output_html)
        print(f"✔ Carte sauvegardée dans : {output_html}")

    return m

Dans la cellule suivante vous pourrez mettre votre CSV modifié.

In [ ]:
uploaded = files.upload()

Et là vous obtiendrez une nouvelle carte.

In [ ]:
filtered_csv = "map_data_la_filtered.csv"
m_filtered = make_map_from_csv(filtered_csv, output_html="map_latin_filtered.html")

Et mieux encore, vous allez pouvoir adapter votre CSV à un outil que je vous recommande (et que nous verrons si nous avons le temps), CARTO.

Vous trouverez tout ce qu'il faut pour CARTO [ici](https://docs.carto.com/faqs/carto-for-education). C'est un outil hyper pratique et très utile si vous faites de la visualisation précise.

In [ ]:
import pandas as pd

df = pd.read_csv("map_data_la.csv")

df_carto = df.rename(columns={
    "lat": "latitude",
    "lon": "longitude",
    "loc_lemma": "name"
})

df_carto.to_csv("map_data_la_carto.csv", index=False, encoding="utf-8")
print("CSV prêt pour CARTO : map_data_la_carto.csv")